In [6]:
from __future__ import annotations
import numpy as np
from qtpyt.block_tridiag import greenfunction
from qtpyt.base.selfenergy import DataSelfEnergy as BaseDataSelfEnergy
from qtpyt.projector import expand
import pickle

In [19]:
class DataSelfEnergy(BaseDataSelfEnergy):
    """Wrapper"""

    def retarded(self, energy):
        return expand(S_molecule, super().retarded(energy), idx_molecule)


def load(filename):
    return DataSelfEnergy(energies, np.load(filename))

def run(outputfile):

    T = np.empty(energies.size)

    for e, energy in enumerate(energies):
        T[e] = gf.get_transmission(energy)

    np.save(outputfile, (energies,T.real))


In [11]:
data_folder = '../output/compute_run'
index_active_region = np.load(f"{data_folder}/index_active_region.npy")
self_energy = np.load(f"{data_folder}/self_energy.npy",allow_pickle=True)
dmft_sigma_file = f"{data_folder}/dmft_sigma.npy"

with open(f"{data_folder}/hs_list_ii.pkl", 'rb') as f:
    hs_list_ii = pickle.load(f)
with open(f"{data_folder}/hs_list_ij.pkl", 'rb') as f:
    hs_list_ij = pickle.load(f)

nodes = [0,810,1116,1278,1584,2394]
# Define energy range and broadening factor for the Green's function calculation
de = 0.2
energies = np.arange(-3., 3. + de / 2., de).round(7)
eta = 1e-3

# Transmission function calculation
imb = 2  # index of molecule block from the nodes list
S_molecule = hs_list_ii[imb][1]  # overlap of molecule
idx_molecule = index_active_region - nodes[imb]  # indices of active region w.r.t molecule


In [15]:
# Initialize the Green's function solver with the tridiagonalized matrices and self-energies
gf = greenfunction.GreenFunction(hs_list_ii,
                                hs_list_ij,
                                [(0, self_energy[0]),
                                (len(hs_list_ii) - 1, self_energy[1])],
                                solver='dyson',
                                eta=eta)


In [20]:
# Transmission function for DFT
# ! Note that here the dft transmission is computed from a Lowdin orthogonalized H and S as starting point. 
# It may be more appropriate to compute it using the non-Lowdin orthogonalized one as shown in the dft_transmission_*.ipynb notebook for more faithful results
outputfile = f"{data_folder}/dft_transmission.npy"
run(outputfile)

In [21]:
# Initialize the Green's function solver with the tridiagonalized matrices and self-energies
gf = greenfunction.GreenFunction(hs_list_ii,
                                hs_list_ij,
                                [(0, self_energy[0]),
                                (len(hs_list_ii) - 1, self_energy[1])],
                                solver='dyson',
                                eta=eta)

# Add the DMFT self-energy for transmission

self_energy[2] = load(dmft_sigma_file)
gf.selfenergies.append((imb, self_energy[2]))

# Transmission function with DMFT
outputfile = f"{data_folder}/dmft_transmission.npy"
run(outputfile)
gf.selfenergies.pop()

(2, <__main__.DataSelfEnergy at 0x1553d9b75970>)